# **Step1: Install amd import libraries**

In [1]:
!pip install -qq "transformers==4.35" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" "tiktoken"
!pip install -qq gradio
!pip install -qq gtts
import pandas as pd
import gradio as gr
from gtts import gTTS
import os
import torch
from datasets import Dataset, load_dataset
from random import randrange
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import torch
from peft import AutoPeftModelForCausalLM
import transformers

print(transformers.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device ',device,' is being used')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


4.35.0
Device  cuda  is being used


# **Step 2: Load Saved Model**

In [2]:
# Model_3
! gdown 17Z8n0X53ho-C_vcc1m2bKanf8pwjt_dw
print(' If you cannot download dataset: please use this link:\n https://drive.google.com/file/d/17Z8n0X53ho-C_vcc1m2bKanf8pwjt_dw/view?usp=sharing')


Downloading...
From (original): https://drive.google.com/uc?id=17Z8n0X53ho-C_vcc1m2bKanf8pwjt_dw
From (redirected): https://drive.google.com/uc?id=17Z8n0X53ho-C_vcc1m2bKanf8pwjt_dw&confirm=t&uuid=0abceaca-a6af-45fc-8ad3-dab6d3e3c8ec
To: /content/Final_Laama_Model.zip
100% 1.60G/1.60G [00:11<00:00, 136MB/s]
 If you cannot download dataset: please use this link:
 https://drive.google.com/file/d/17Z8n0X53ho-C_vcc1m2bKanf8pwjt_dw/view?usp=sharing


In [3]:
!unzip -qq '/content/Final_Laama_Model.zip'

In [4]:
model_directory = '/content/results/lamma/checkpoint-2100' # Set this to your directory

new_model = AutoPeftModelForCausalLM.from_pretrained(
    model_directory,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

##2-1 Load tokenizer

In [5]:

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_directory, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# **Step 3: Generate answer**

In [6]:
def answer_query(query,temperature=0.6,long_answer = False):

    if long_answer:
      prompt= 'Generatee long answer for this question ' + query

    else:
      prompt= 'Generate short answer for this question ' + query

    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()

    if long_answer:

        outputs = new_model.generate(input_ids=input_ids,


                             min_length=100,
                             max_new_tokens=300,
                                 #repetition_penalty=0.2,
                            #  do_sample=True,
                            #  top_p=0.9,
                             temperature=temperature)

    else:

        outputs = new_model.generate(input_ids=input_ids,


                             min_length=40,
                             #max_length = 100,
                             max_new_tokens=70,
                                 #repetition_penalty=0.2,
                            #  do_sample=True,
                            #  top_p=0.9,
                             temperature=temperature)
    answer = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

    return prompt,answer

## 3-1 Make answer pretty

In [7]:
# Remove question from answer and remove last uncomplete sentence
def pretty_asnwer(prompt,answer):
    new_answer = answer
    if new_answer.endswith('.') or new_answer.endswith('!') or new_answer.endswith('?'):

        last_index_of_question = len(prompt)
        new_answer = new_answer[last_index_of_question:]
        if new_answer.find('Answer')>-1:
          index = new_answer.find('Answer')
          new_answer = new_answer[index+8:]
        #print(new_answer)

    else:
        #print('Not complete answer')
        last_index_of_question = len(prompt)
        new_answer = new_answer[last_index_of_question:]
        last_sentence = new_answer.split('. ')[-1]
        new_answer = new_answer.replace(last_sentence, " ")
        if new_answer.find('Answer')>-1:
          #print('here')
          index = new_answer.find('Answer')
          new_answer = new_answer[index+8:]



    return new_answer



# **Step 4: Ueser Interface**

In [8]:

# Check availabilty:
def disability(query,long_answer):
  result = 0
  while len(result)<20: # Generate again if, last time model was disable
      answer = answer_query(query,long_answer=long_answer)
      new_answer = pretty_asnwer(answer)
      result = new_answer

  return result

def text_to_speech(text):
        # Convert text to speech
        tts = gTTS(text)
        # Save the converted audio to a file
        tts.save("output.mp3")
        # Return the file path
        return "output.mp3"

def aks_question(Question, Long_Format, Short_Format, Audio_Play ):
    query = Question

    #check Long format

    if Long_Format == True and Short_Format == True:
        # query_copy_1 = query
        # query_copy_2 = query

        prompt,long_answer = answer_query(query,temperature=0.6,long_answer = True)
        long_answer = pretty_asnwer(prompt,long_answer)

        while len(long_answer)<50: # Generate again if, last time model was disable
            prompt,long_answer = answer_query(query,temperature=0.6,long_answer = True)
            long_answer = pretty_asnwer(long_answer)


        prompt,short_answer = answer_query(query,temperature=0.6,long_answer = False)
        short_answer = pretty_asnwer(prompt,short_answer)


        while len(short_answer)<30: # Generate again if, last time model was disable
          prompt,short_answer = answer_query(query,temperature=0.6,long_answer = False)
          short_answer = pretty_asnwer(prompt,short_answer)

        if Audio_Play:
          text = 'Long answer:'+ long_answer + 'Short answer:'+short_answer
          audio_ouput = text_to_speech(text)
          return long_answer, short_answer, audio_ouput

        else:
          text = ' You did not chose audio play'
          audio_ouput = text_to_speech(text)
          return long_answer, short_answer, audio_ouput


    elif Long_Format == True and Short_Format == False:
        prompt,long_answer = answer_query(query,temperature=0.6,long_answer = True)
        long_answer = pretty_asnwer(prompt,long_answer)

        while len(long_answer)<50: # Generate again if, last time model was disable
            prompt,long_answer = answer_query(query,temperature=0.6,long_answer = True)
            long_answer = pretty_asnwer(long_answer)


        if Audio_Play:
          text = 'Long answer:'+ long_answer
          audio_ouput = text_to_speech(text)
          return long_answer, ' ', audio_ouput

        else:
          text = ' You did not chose audio play'
          audio_ouput = text_to_speech(text)
          return long_answer, ' ', audio_ouput


    elif Long_Format == False and Short_Format == True:
        prompt,short_answer = answer_query(query,temperature=0.6,long_answer = False)
        short_answer = pretty_asnwer(prompt,short_answer)

        while len(short_answer)<30: # Generate again if, last time model was disable
          prompt,short_answer = answer_query(query,temperature=0.6,long_answer = False)
          short_answer = pretty_asnwer(prompt,short_answer)

        if Audio_Play:
          text = 'short answer:'+ short_answer
          audio_ouput = text_to_speech(text)
          return ' ',short_answer , audio_ouput

        else:
          text = ' You did not chose audio play'
          audio_ouput = text_to_speech(text)
          return ' ', short_answer, audio_ouput


    else:
        text = ' Please check one checkbox at least'
        audio_ouput = text_to_speech(text)
        return 'Please check one checkbox at least', 'Please check one checkbox at least', audio_ouput


demo = gr.Interface(
    fn=aks_question,

    inputs=["text", "checkbox","checkbox","checkbox"],

    outputs=[
        gr.Textbox(label="Long Answer"),
        gr.Textbox(label="Short Answer"),
        gr.Audio(label='Long_Format_Audio_Play')
    ]
)
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://62a6645e8a2256df55.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
